# Funções para facilitar manipulação de dados

In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader
from surprise import SVD, SVDpp, accuracy
from collections import defaultdict
import random
import os

    

path = r'C:\Users\Elias-Acer\DH\Projeto\KASSANDR\Samples\2.Clicks\clicks_de_sample.txt'

def carrega_dataset(path):
    df = pd.read_csv(path)
    lista = []
    count = 0
    lista_ids = []


    for coluna in df.UserId.unique():
        #print(coluna,'_',count)
        aux = 'user_' + str(count)
        #aux = count
        #print(aux.strip())
        lista_ids.append(coluna)
        lista.append(aux)
        count += 1

    lista_offer = []
    count = 0
    lista_offer_ids = []

        #renomeando as ofertas em oferta_1, oferta_2, etc...
    for coluna in df.OfferId.unique():
        #print(coluna,'_',count)
        aux = 'oferta_' + str(count)
        #aux = count
        #print(aux.strip())
        lista_offer_ids.append(coluna)
        lista_offer.append(aux)
        count += 1

    dici_usuarios = dict(zip(lista_ids, lista))
    dici_ofertas = dict(zip(lista_offer_ids, lista_offer))
    df['User'] = df['UserId'].map(dici_usuarios)
    df['Offer'] = df['OfferId'].map(dici_ofertas)

    return df


def corta_dataset(df, thr):
    

    clicks_usuario =  pd.DataFrame(df.User.value_counts()).reset_index()
    clicks_usuario.columns = ['User','clicks']
    clicks_usuario = dict(zip(clicks_usuario.User, clicks_usuario.clicks))
    df['clicks_usuario'] = df['User'].map(clicks_usuario)

    mask3 = df.clicks_usuario > thr
    df_mod = df.loc[mask3, :]
    
    return df_mod

def seleciona_usuario_corta_categorias(df, user):
    
    categorias_usuario = df.loc[df['User'] == user, 'Category'].unique()
    mask = df['Category'].isin(categorias_usuario)
    df_semcategorias = df.loc[~mask]
    
    return df_semcategorias

def cria_sparsa(df):
    sparse = df.pivot_table(values='clicks_usuario',index='User',columns='Offer')

    sparse[sparse.isna()] = 0
    sparse[sparse != 0] = 1
    
    return sparse

def cria_melted(sparse): #recebe a sparse
    return pd.melt(frame = sparse, var_name = 'Offer', value_name = 'Clicked', ignore_index = False)


In [2]:
def reader_data(melted):
    
    melted.reset_index(inplace = True)
    reader = Reader(rating_scale = (0,1))
    data = Dataset.load_from_df(df = melted, reader = reader)
    
    return data



def prepara_svd_svdpp(df,melted, user_id, n_recommendations):

    data = reader_data(melted)

    svd = SVD()
    
    #svdpp = SVDpp()

    svd.fit(data.build_full_trainset())
    
    
    mask = melted.index == user_id #fazer a negativa
    list_of_unclicked_offer = melted.loc[~mask].reset_index()
    list_of_unclicked_offer = list_of_unclicked_offer['Offer'].unique()
    
    user_set = [[user_id, item_id, 0] for item_id in list_of_unclicked_offer]

    algo = svd
    predictions = algo.test(user_set)

    top_n_recommendations = defaultdict(list)

    for user_id, item_id, _, est, _ in predictions:

        top_n_recommendations[user_id].append((item_id, est))


    for user_id, ratings in top_n_recommendations.items():

        ratings.sort(key=lambda x: x[1], reverse=True)

        top_n_recommendations[user_id] = ratings[:n_recommendations]


    count = 0

    print("Recommendations for user with id {}: ".format(user_id))

    for movie_index, score in top_n_recommendations[user_id]: 

        count +=1

        print('{}. {}, predicted rating = {}'.format(count, df[df['Offer']==movie_index]['OfferTitle'].iloc[0], round(score,3)))


In [3]:
%%time
df = corta_dataset(carrega_dataset(path), 40)
df['User'].unique()

Wall time: 13.4 s


array(['user_141', 'user_758', 'user_1310', 'user_1927', 'user_2422',
       'user_2738', 'user_4880', 'user_5318', 'user_5456', 'user_5683',
       'user_5735', 'user_5830', 'user_7285', 'user_7639', 'user_8468',
       'user_10381', 'user_12261', 'user_14290', 'user_15169',
       'user_16402', 'user_16853', 'user_18624', 'user_18895',
       'user_26901', 'user_26919', 'user_29360', 'user_29742',
       'user_30445', 'user_31763', 'user_33226', 'user_35878',
       'user_39659', 'user_39963', 'user_40342', 'user_40812',
       'user_42608', 'user_47168', 'user_49991', 'user_51237',
       'user_52497', 'user_55138', 'user_57009', 'user_63761',
       'user_65544', 'user_65818', 'user_77684', 'user_78835',
       'user_79212', 'user_79712', 'user_79978', 'user_83455',
       'user_84373', 'user_84475', 'user_85463', 'user_85684',
       'user_87592', 'user_89639', 'user_94949', 'user_97517',
       'user_110100', 'user_110403', 'user_127857', 'user_140478',
       'user_148781', 'use

In [4]:
usuario = random.choice(df['User'].unique())
usuario

'user_83455'

In [5]:
df_t = seleciona_usuario_corta_categorias(df, usuario)
df_t

,UserId,OfferId,OfferViewId,CountryCode,Category,Source,UtcDate,Keywords,OfferTitle,User,Offer,clicks_usuario
142,32d3feacea939ce19c012c4fffa216a4d52c4dcfbe6205...,3fa034f49aba80b0ab64696616cd7f6d,3fa034f49aba80b0ab64696616cd7f6d-1076982529341...,de,100354123,de08e3066ad92af6163c1759104a0e8487f39405d64713...,2016-06-02 00:15:26.0,NaN,Lada Niva 4x4 Taiga - ABS- Euro5-sofort verfügbar,user_141,oferta_140,55
767,9a2987f649b7abbff1ab52d2e43c3057e6440a5fa2242b...,65c3bd386cd23007d35683ab9c811e5b,65c3bd386cd23007d35683ab9c811e5b-1076982502492...,de,100477023,7ecf85d55cde49eff1b5b9f4f24fe716f91252607cab3c...,2016-06-01 19:49:08.0,NaN,Brille24 Veria in anthrazit/schwarz,user_758,oferta_734,61
1984,7178d0466f56a65c715e76d7074ecd969bc22bcced317d...,93a15927992ef907afd7b4177d8f0092,93a15927992ef907afd7b4177d8f0092-1076981391636...,de,137001,0cc5e761ab6bcf93922f3a419f96c89359a77da0e4afe8...,2016-06-01 14:01:10.0,NaN,Maintal Boxspringbett,user_1927,oferta_1821,45
2510,86feaf5a9b88c9b27ddad9f936a06ce3ffd927e5ce9702...,2fda4f6b0d88eb57f6c43d9c46e900a9,2fda4f6b0d88eb57f6c43d9c46e900a9-1076981212888...,de,100354123,7ffbeedfb14f46f259001cefc4d79048ccef25b9e87207...,2016-06-01 09:27:10.0,NaN,Dacia Duster dCi 110 FAP 4x4 Prestige *origina...,user_2422,oferta_2282,42
2849,0130c614866313a1f0baedd9a50db9f9ba110f25af1ce9...,b14e64081f08df297a75d710968f3b51,b14e64081f08df297a75d710968f3b51-1076982532955...,de,100568013,f7151fb3a6269230dcaa048646568c53ad69ab3b3d24ae...,2016-06-01 16:00:50.0,NaN,Ruffwear Singletrak Pack Cloudburst Gray Brust...,user_2738,oferta_2575,57
...,...,...,...,...,...,...,...,...,...,...,...,...
695572,505df99d47a21759714bc6bbb7c2793bfbe903f57aa8c3...,a71f4971f48087ed5629125cdda20738,a71f4971f48087ed5629125cdda20738-null,de,100291623,5785c0ceb5dc05e67e2d1a4856d6e8883cdf250cfc3a41...,2016-06-30 06:56:28.0,NaN,Yves Rocher Nail-Art Sticker POPEXOTIC,user_8468,oferta_323132,79
695578,9fcb6472a6f2cd727de9c00aad801d94288380f5f5f4e6...,9de03ec8bb3b9eedd3c4668542975506,9de03ec8bb3b9eedd3c4668542975506-107698418633_...,de,100441023,a4163e1501c4e3fc5fef534c7fd15fc9df5802659acc01...,2016-06-30 11:44:29.0,NaN,LIEDECO Aluminium-Jalousie Jalousie aus Alumin...,user_77684,oferta_325731,42
695620,e119fb058c279feaf683b4fee1234f26da91b5cf22560f...,c3bf9da5b139822839eb777b229ab7dd,c3bf9da5b139822839eb777b229ab7dd-1076982508147...,de,117001,a4163e1501c4e3fc5fef534c7fd15fc9df5802659acc01...,2016-06-30 11:12:03.0,NaN,AVG Protection 2016,user_18624,oferta_75236,41
696360,bcedb937ac892fe1041a7f00185544c14cc4237990d89b...,04b9905ef20967d9070f89e2bf297a37,04b9905ef20967d9070f89e2bf297a37-107698411522_...,de,164401,22bb70d5745725c3f8bf9bd3cf83ffc078ec126eda1567...,2016-06-30 12:28:40.0,NaN,Opinel Messer Nature Buchsbaum Motivgravur mit...,user_79712,oferta_207754,42


In [6]:
sparse = cria_sparsa(df_t)
sparse

Offer,oferta_100182,oferta_100196,oferta_100357,oferta_100443,oferta_100506,oferta_100554,oferta_100772,oferta_100796,oferta_100875,oferta_101126,...,oferta_98577,oferta_98872,oferta_98999,oferta_99040,oferta_99042,oferta_99045,oferta_99155,oferta_99484,oferta_9986,oferta_9987
User,,,,,,,,,,,,,,,,,,,,,
user_10381,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_110100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
user_110403,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
user_12261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_127857,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
user_85684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_87592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_89639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
%%time
melted = cria_melted(sparse)
melted

Wall time: 20.9 ms


,Offer,Clicked
User,,
user_10381,oferta_100182,1.0
user_110100,oferta_100182,0.0
user_110403,oferta_100182,0.0
user_12261,oferta_100182,0.0
user_127857,oferta_100182,0.0
...,...,...
user_85684,oferta_9987,0.0
user_87592,oferta_9987,0.0
user_89639,oferta_9987,0.0


In [8]:
%%time
user_id = usuario

prepara_svd_svdpp(df_t, melted, user_id, 10)

Recommendations for user with id user_83455: 
1. Renault Laguna II Exception, predicted rating = 0.24
2. Subaru WRX STI Sport TOP neuwertig, predicted rating = 0.225
3. Audi Q5 2.0 TDI QUATTRO LEDER NAVI+ XENON+, predicted rating = 0.217
4. Mercedes-Benz A 140 KLIMAANLAGE, predicted rating = 0.21
5. Audi SQ5 3.0 TDI Quattro Pano, predicted rating = 0.201
6. Ligier Nova 45KM/H MOPEDAUTO - LEICHTMOBIL AB 16JAHRE, predicted rating = 0.193
7. Ligier X-TOO RS 45 Km/h Kostenlose Anlieferung, predicted rating = 0.188
8. Porsche 996 911 996 Carrera 4S, predicted rating = 0.186
9. Skoda Yeti Skoda Yeti 1.4 TSI Ambition PLUS EDITION, predicted rating = 0.185
10. Kia Sportage 1.7 CRDI CD AUX MP3 ESP Radio Klimaautom, predicted rating = 0.185
Wall time: 25.8 s


## Italia

In [9]:
#testando com outros paises

path_it = r'C:\Users\Elias-Acer\DH\Projeto\KASSANDR\Samples\2.Clicks\clicks_it_sample.txt'

df = corta_dataset(carrega_dataset(path_it), 40)
df['User'].unique()

array(['user_28', 'user_784', 'user_828', 'user_870', 'user_946',
       'user_1072', 'user_1611', 'user_2017', 'user_2371', 'user_2762',
       'user_3470', 'user_4471', 'user_7704', 'user_7888', 'user_7952',
       'user_8388', 'user_9161', 'user_10138', 'user_10743', 'user_11385',
       'user_12246', 'user_14523', 'user_15427', 'user_17405',
       'user_19240', 'user_20757', 'user_21392', 'user_21437',
       'user_21559', 'user_22664', 'user_22717', 'user_22738',
       'user_26078', 'user_31692', 'user_33272', 'user_34015',
       'user_35659', 'user_45265', 'user_45914', 'user_47019',
       'user_50816', 'user_53840', 'user_56077', 'user_58328',
       'user_59367', 'user_62754', 'user_63809', 'user_64904',
       'user_66247', 'user_67725', 'user_67906', 'user_75794',
       'user_78701', 'user_78765', 'user_79189', 'user_79726',
       'user_79837', 'user_79911', 'user_80413', 'user_84586',
       'user_89221', 'user_91144', 'user_95838', 'user_99365',
       'user_102023', 

In [10]:
usuario = random.choice(df['User'].unique())
usuario

'user_102905'

In [11]:
df_t = seleciona_usuario_corta_categorias(df, usuario)
df_t

,UserId,OfferId,OfferViewId,CountryCode,Category,Source,UtcDate,Keywords,OfferTitle,User,Offer,clicks_usuario
793,29619b8f6e2dd42f3f2e552dab73db7235155859d5839f...,d796a1f63f5d685c93f6b11931fbeb38,d796a1f63f5d685c93f6b11931fbeb38-1076982529717...,it,100358623,88e955ce0b1cfb7d962115f5ded0cdc0a42533bf10847a...,2016-06-02 00:07:52.0,NaN,C Rhodia Clairefontaine,user_784,oferta_752,300
838,34eda4c41513683056f23f19d93796919a2b7a9d6e40cb...,be0558f8b86d0d31ab44c350f548d044,be0558f8b86d0d31ab44c350f548d044-1076982042304...,it,100317223,8650950282af5698bb33df43a5477d6b25e3ed76817e1a...,2016-06-01 09:49:00.0,NaN,Abito Sposa In Raso Bicolore,user_828,oferta_795,44
880,4cbb82d0177fb7edd18b3eb0636986bbc4131da23a52a1...,fcbc2c07b7ae15ddabe7925dd41c0446,fcbc2c07b7ae15ddabe7925dd41c0446-null,it,100311823,231438396d018c27abf9402f8267529c497f7e0a24f888...,2016-06-01 15:19:02.0,NaN,Miia MTV-D19LEHD TV 19 pollici HD Ready DVBT V...,user_870,oferta_833,99
959,131e39debb74b154dedc24e307c357de5cf8f98f34b7ab...,ee9ab56d78145ded9c682868f1759313,ee9ab56d78145ded9c682868f1759313-1076982532973...,it,100126013,96c052cfa064e92d4b06913f1f67a20535931e51e2f204...,2016-06-01 21:56:11.0,NaN,FRESH&#39;N REBEL 1rb3000pt Verde Acqua,user_946,oferta_902,42
1087,0caeb5240dee7c458b4ac95215d3b8532b15ca598256d9...,540645b221a5826d4ccef002b15e5a6d,540645b221a5826d4ccef002b15e5a6d-1076982511991...,it,100356623,88e955ce0b1cfb7d962115f5ded0cdc0a42533bf10847a...,2016-06-01 20:43:06.0,alan ricetrasmittente,Radiorama The Legend,user_1072,oferta_1018,163
...,...,...,...,...,...,...,...,...,...,...,...,...
999447,cfb09f96017ca0cf8a8112e275b8d195c52685dd2b2743...,a344b943ef5d808ff7775d0709a456d3,a344b943ef5d808ff7775d0709a456d3-1076984115966...,it,116501,299c716ce9d99bb8f968596f0090744256bb47064de12b...,2016-06-16 21:49:34.0,NaN,PNY Ssd Cs 1111 480gb,user_2371,oferta_310641,97
999522,950481cf4bed3858e1dbb639ad38a59b913df2de70f9b4...,fe32b9bdf2f84d9e5ff5b4ea2ef5d18a,fe32b9bdf2f84d9e5ff5b4ea2ef5d18a-1076984115966...,it,100020213,38cece1fbcfcb79b6606dea6c12ad16f68d924543c72c8...,2016-06-16 20:43:17.0,NaN,Huawei Y6 Smartphone,user_20757,oferta_194966,65
999550,c5103968e5ff140dd7ac9e8783a5e66267155f9608d1af...,581f90b853eceae2493cce38226c571f,581f90b853eceae2493cce38226c571f-null,it,100566113,5483c7740fe263d9d83a149ab02e4484c9e056b6520c16...,2016-06-16 02:18:38.0,NaN,Portabottiglie 15 sedi,user_388782,oferta_316664,75
999825,40c3b349c711a0a581eb246115a089c9dc23e2ef1a8700...,af29196947cd14b932f8ad5ad8f922cf,af29196947cd14b932f8ad5ad8f922cf-null,it,100311823,8bad96f489caf83456d6392c506ddd556d33b81c10da7b...,2016-06-16 22:18:53.0,NaN,LG 65UF850v TV LED 65&quot; UHD 4k 3D Smart TV...,user_458311,oferta_5698,45


In [12]:
sparse = cria_sparsa(df_t)
sparse

Offer,oferta_100052,oferta_100070,oferta_1002,oferta_100249,oferta_100255,oferta_100287,oferta_100317,oferta_100332,oferta_10037,oferta_100397,...,oferta_99649,oferta_99690,oferta_99705,oferta_99709,oferta_99810,oferta_99860,oferta_99890,oferta_99893,oferta_99954,oferta_99965
User,,,,,,,,,,,,,,,,,,,,,
user_10138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_102023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_102237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_102269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_102437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
user_91144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_9161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
%%time
melted = cria_melted(sparse)
melted

Wall time: 158 ms


,Offer,Clicked
User,,
user_10138,oferta_100052,0.0
user_102023,oferta_100052,0.0
user_102237,oferta_100052,0.0
user_102269,oferta_100052,0.0
user_102437,oferta_100052,0.0
...,...,...
user_91144,oferta_99965,0.0
user_9161,oferta_99965,0.0
user_946,oferta_99965,0.0


In [14]:
%%time
user_id = usuario
prepara_svd_svdpp(df_t, melted, user_id, 10)

Recommendations for user with id user_102905: 
1. Festive Productions Idee Brite gestito bambola gonfiabile batteria 42 centimetri di neve costume bambino, predicted rating = 0.064
2. Dcg MF1100C Mini Frigorifero Potenza 49 Watt Capacità 20 Litri, predicted rating = 0.057
3. Paderno 48294-01 Mantecatore per Burro, predicted rating = 0.052
4. Frankenstolz Materasso a molle insacchettate &quot;Happy T&quot; 7 zone grado di durezza: 2, predicted rating = 0.05
5. Fujifilm x-t1 nero + 18-55mm f2.8-4 r lm ois - subito disponibile - garanzia 24 mesi in italia, predicted rating = 0.047
6. C Rhodia Apik Sous-Main Jdo, predicted rating = 0.047
7. RUCO LINE Sneakers & Tennis shoes alte Donna, predicted rating = 0.046
8. Annette Schmitt Australian Shepherd, predicted rating = 0.046
9. Abito Da Cerimonia In Chiffon, predicted rating = 0.044
10. Sony HDR-AZ1VR Action Camera con Kit Telecomando Live View, predicted rating = 0.044
Wall time: 3min 5s


## França

In [15]:
#testando com outros paises

path_it = r'C:\Users\Elias-Acer\DH\Projeto\KASSANDR\Samples\2.Clicks\clicks_fr_sample.txt'

df = corta_dataset(carrega_dataset(path_it), 40)
df['User'].unique()

array(['user_323', 'user_989', 'user_1509', 'user_1628', 'user_2079',
       'user_4419', 'user_4945', 'user_5734', 'user_6148', 'user_8856',
       'user_10059', 'user_11349', 'user_11521', 'user_11989',
       'user_12261', 'user_12626', 'user_12656', 'user_12881',
       'user_13355', 'user_13891', 'user_14868', 'user_15254',
       'user_16213', 'user_16359', 'user_16406', 'user_16873',
       'user_17663', 'user_17920', 'user_20546', 'user_20672',
       'user_22127', 'user_22359', 'user_22747', 'user_22760',
       'user_24606', 'user_25542', 'user_27804', 'user_29808',
       'user_30119', 'user_31903', 'user_31944', 'user_32474',
       'user_32880', 'user_33591', 'user_33611', 'user_35348',
       'user_40443', 'user_40598', 'user_41266', 'user_41594',
       'user_46865', 'user_47585', 'user_49129', 'user_52528',
       'user_52767', 'user_54576', 'user_54961', 'user_56000',
       'user_56533', 'user_56815', 'user_57933', 'user_58346',
       'user_58730', 'user_60128', 'use

In [16]:
usuario = random.choice(df['User'].unique())
usuario

'user_161149'

In [17]:
df_t = seleciona_usuario_corta_categorias(df, usuario)
df_t

,UserId,OfferId,OfferViewId,CountryCode,Category,Source,UtcDate,Keywords,OfferTitle,User,Offer,clicks_usuario
324,719a882798267724db9921c63ba000a2ba59124b8ce2e8...,46c1564caf38b0db80ed5e7e24353378,46c1564caf38b0db80ed5e7e24353378-1076982042256...,fr,127701,676828ddbf483b80dabb48af6d318a3162dc5cc16fbb0c...,2016-06-01 19:19:51.0,NaN,Takara VR 139 Lecteur DVD portable écran 9&quo...,user_323,oferta_320,91
997,527ee8220d94ea8d0a501a835f0dca117c65a2a8c51457...,37b9d5bd2d7508ef1f89cbbb24a3bd55,37b9d5bd2d7508ef1f89cbbb24a3bd55-1076982042256...,fr,100093313,c198732146974db69539a652bc49194793adfe98571a33...,2016-06-01 10:50:26.0,NaN,RICHEL Serre tunnel maraîchère - 18 m²,user_989,oferta_623,274
1524,18ff4116673892de0952de1d955be16b3e1d2a36102c5d...,eab6c4e79465bac903cf8f4439fb52a3,eab6c4e79465bac903cf8f4439fb52a3-1076982502303...,fr,124901,aeb65497e3f81a83f416f07848d37fe44d791f51654708...,2016-06-01 21:16:38.0,NaN,NIKON D3300 - Appareil photo numerique + objec...,user_1509,oferta_1453,41
1554,527ee8220d94ea8d0a501a835f0dca117c65a2a8c51457...,1c00ce8636dcd4ca48d32a950a33be2c,1c00ce8636dcd4ca48d32a950a33be2c-1076982532925...,fr,114401,c198732146974db69539a652bc49194793adfe98571a33...,2016-06-01 10:37:31.0,NaN,HP Ecran HP LA2006XB,user_989,oferta_1482,274
1764,527ee8220d94ea8d0a501a835f0dca117c65a2a8c51457...,37b9d5bd2d7508ef1f89cbbb24a3bd55,37b9d5bd2d7508ef1f89cbbb24a3bd55-1076982042256...,fr,100093313,c198732146974db69539a652bc49194793adfe98571a33...,2016-06-01 10:49:51.0,NaN,RICHEL Serre tunnel maraîchère - 18 m²,user_989,oferta_623,274
...,...,...,...,...,...,...,...,...,...,...,...,...
998138,3f75dd9ce92e9bd5ddee95dae198784a78d05b8e6c7e43...,0f959acf265f4185edc629fce1fb7ed3,0f959acf265f4185edc629fce1fb7ed3-null,fr,110601,cfacb1a7a11881514eac86d9c00e2985c347f6403c1fe5...,2016-06-16 11:55:57.0,NaN,Logitech Logi z623 speaker system 2 1 black en...,user_144680,oferta_82264,43
998587,9b5d8c74b571293ba76b178592993508ec42d3329fef5d...,6ccf217cc83a604bf85d4fe6bae662cf,6ccf217cc83a604bf85d4fe6bae662cf-1076982532480...,fr,100010113,6272c77f46f068bbd7ff791a927e3890a884defa689d81...,2016-06-16 08:08:17.0,asus x750jb ty031h,Thomann DJ Bag 80 LPs,user_32880,oferta_141058,66
998706,8dc23b37c03b93899fca5aae6f3a0ecf5c75f82b07d391...,5364850e8341afa32d010bd69039a1be,5364850e8341afa32d010bd69039a1be-1076982522377...,fr,109301,4e711361822ccdc89f2c1a653eaa16ddabff875a6d1854...,2016-06-16 23:20:50.0,NaN,Palladium Bottines et boots Palladium( Homme ),user_58346,oferta_86222,48
999357,719a882798267724db9921c63ba000a2ba59124b8ce2e8...,a7d5eeedc641d847f4f80b57a9dadcdb,a7d5eeedc641d847f4f80b57a9dadcdb-1076982503240...,fr,146301,eee20b447d04735c40276240146e6e44d0cbcb0f881de4...,2016-06-16 23:25:58.0,NaN,Brandt FC265MHW Mini Four 26 L,user_323,oferta_18562,91


In [18]:
sparse = cria_sparsa(df_t)
sparse

Offer,oferta_10007,oferta_100098,oferta_100240,oferta_10031,oferta_10035,oferta_100360,oferta_100566,oferta_100609,oferta_100613,oferta_100705,...,oferta_99372,oferta_99591,oferta_99594,oferta_9967,oferta_99719,oferta_99839,oferta_99895,oferta_9990,oferta_99932,oferta_99937
User,,,,,,,,,,,,,,,,,,,,,
user_10059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_102896,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_110526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_11349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_114487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
user_97085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_97943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
user_98239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
%%time
melted = cria_melted(sparse)
melted

Wall time: 87 ms


,Offer,Clicked
User,,
user_10059,oferta_10007,0.0
user_102896,oferta_10007,0.0
user_110526,oferta_10007,0.0
user_11349,oferta_10007,0.0
user_114487,oferta_10007,0.0
...,...,...
user_97085,oferta_99937,0.0
user_97943,oferta_99937,0.0
user_98239,oferta_99937,0.0


In [20]:
%%time
user_id = usuario
prepara_svd_svdpp(df_t, melted, user_id, 10)

Recommendations for user with id user_161149: 
1. Vivid arts XRL-HERM-B Tortue d&#39;Hermann, predicted rating = 0.185
2. Vilmorin 1167851 Pack de Graines Haricot Triomphe de Farcy, predicted rating = 0.144
3. Bushnell 119756C Panneau solaire Marron, predicted rating = 0.142
4. Fischer safety two remorque de vélo pour enfant taille unique bleu 86384, predicted rating = 0.133
5. HOPPE poignée de porte avec rosette BB 2950616 Verona argent anodisé, predicted rating = 0.132
6. Polti Aspirateur Nettoyeur Vapeur Vaporetto Lecoaspira FAV70 Intelligence , predicted rating = 0.125
7. LENOVO Ordinateur portable 14&#39;&#39; I3 4Go 500Go Win10 U41-70 Rouge, predicted rating = 0.121
8. Gre echelle piscine es1150, predicted rating = 0.12
9. Herma Autocollant Bleu, predicted rating = 0.118
10. Targus Sac à dos avec roulettes City Gear pour ordinateur portable de 15&quot; à 15.6&quot; - Noir - TSB700EU, predicted rating = 0.11
Wall time: 1min 7s
